In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from transformers import BitsAndBytesConfig
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
from dotenv import load_dotenv
load_dotenv()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "/content/drive/MyDrive/llama_finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path,token=os.getenv("HF_TOKEN"))
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto",load_in_8bit=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [8]:
def generate_response(prompt, max_new_tokens=150):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

sample_prompts = [
    "what is the best crop to grow in firozabad",
    "is wheat suitable for indore and best fertilizer to use?",
    "best crop to grow in sambalpur odissa?"
]

for prompt in sample_prompts:
    print(f"🟢 Prompt: {prompt}")
    print(f"🔹 Response: {generate_response(prompt)}\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🟢 Prompt: what is the best crop to grow in firozabad


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🔹 Response: what is the best crop to grow in firozabad district of uttar pradesh given the soil has nitrogen: 66.11190029, phosphorus: 67.01101064, potassium level are deficient, and pH of the soil is 6.354439457?Based on the provided conditions, the best Crop for the region Firozabād District, Uttar Pradēsh is wheat. Preferably Jau or Whole Wheat Given the specified conditions, Growing jau (wheat) will be more beneficial as it can tolerate lower Nitrogen levels better than other crops..Note: This answer is based on the decision tree algorithm for recommending crops based on Soil Conditions. For a more accurate result, consider consulting local experts or conducting further research specific to your area..
The

🟢 Prompt: is wheat suitable for indore and best fertilizer to use?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🔹 Response: is wheat suitable for indore and best fertilizer to use? for the given conditions, Wheat: Suitable region - Temperate regions with moderate rainfall. Preferred soil - Well-drained loamy soils.. Soil pH 5.5-7.0 Ideal temperature - 10°C to 30°C Optimum moisture - Average to high Moisture requirement - High Water stress tolerance -- Low Tolerance to drought Drought tolerance index 2 Suitable crop after - Maize, Peas Given alternative is Ragi: Not preferred under similar conditions.
Answer of the question is Raggi. Since the condition specified requires a low water stressed crop, it should be given as an alternative to such crops. The best answer is Raggi.

🟢 Prompt: best crop to grow in sambalpur odissa?
🔹 Response: best crop to grow in sambalpur odissa? Based on the given criteria, Sembu: Given its high yield and resistance to pests, it is an ideal choice for farmers. It can be grown under various weather conditions and requires minimal water.. The best crop to plant as a cov